#Simple RLHF Implementation by Nandini Lokesh Reddy

In [ ]:
# RLHF Implementation for Text Summarization
# First, install required packages
!pip install -q transformers==4.28.1 datasets==2.12.0 torch==2.0.0 numpy tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Training

In [ ]:
import os
import json
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForCausalLM,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    RobertaForSequenceClassification,
    RobertaTokenizer,
    AdamW
)
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Create sample data if needed
def create_sample_data():
    """Create sample data files for testing."""
    sample_data = [
        {
            "input_text": "I live right next to a huge university, and have been applying for a variety of jobs with them through their faceless electronic jobs portal for a few months.",
            "candidate_0": "When applying through a massive job portal, is just one HR person seeing ALL of them?",
            "candidate_1": "When applying to many jobs through a single university jobs portal, is just one HR person reading ALL my applications?",
            "choice": 1
        },
        {
            "input_text": "I currently live in Texas and I plan on going to university in England, and I think I want to stay there for a while.",
            "candidate_0": "I want to go on a road trip from Texas to England to visit as many places as possible. Which route should I choose?",
            "candidate_1": "How do I plan a road trip in a way that I can see the places I want to see, but also see the places I haven't seen?",
            "choice": 1
        }
    ]

    if not os.path.exists('sample_preference.jsonl'):
        with open('sample_preference.jsonl', 'w') as f:
            for item in sample_data:
                f.write(json.dumps(item) + '\n')
        print("Created sample_preference.jsonl")

    if not os.path.exists('sample_prompt.jsonl'):
        with open('sample_prompt.jsonl', 'w') as f:
            for item in sample_data[:1]:  # Just add one item
                f.write(json.dumps(item) + '\n')
        print("Created sample_prompt.jsonl")

create_sample_data()

# Step 1: Custom Dataset for Preference Data
class PreferenceDataset(Dataset):
    def __init__(self, file_paths, tokenizer, max_length=512):
        self.examples = []
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Load data from JSONL files
        for file_path in file_paths:
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        if line.strip():
                            try:
                                item = json.loads(line)
                                if 'input_text' in item and 'candidate_0' in item and 'candidate_1' in item and 'choice' in item:
                                    self.examples.append(item)
                            except json.JSONDecodeError:
                                print(f"Skipping invalid JSON line in {file_path}")
            else:
                print(f"Warning: File {file_path} not found")

        print(f"Loaded {len(self.examples)} examples")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        item = self.examples[idx]
        prompt = item['input_text']
        chosen_idx = item['choice']
        rejected_idx = 1 - chosen_idx

        # Get chosen and rejected summaries
        chosen = item[f'candidate_{chosen_idx}']
        rejected = item[f'candidate_{rejected_idx}']

        # Prepare inputs for reward model training
        chosen_text = f"Prompt: {prompt}\nSummary: {chosen}"
        rejected_text = f"Prompt: {prompt}\nSummary: {rejected}"

        # Tokenize
        chosen_encodings = self.tokenizer(
            chosen_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        rejected_encodings = self.tokenizer(
            rejected_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Prepare inputs for policy model
        prompt_text = f"Write a concise summary of the following text:\n{prompt}\nSummary:"
        prompt_encodings = self.tokenizer(
            prompt_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "prompt": prompt,
            "chosen": chosen,
            "rejected": rejected,
            "input_ids_chosen": chosen_encodings["input_ids"].squeeze(),
            "attention_mask_chosen": chosen_encodings["attention_mask"].squeeze(),
            "input_ids_rejected": rejected_encodings["input_ids"].squeeze(),
            "attention_mask_rejected": rejected_encodings["attention_mask"].squeeze(),
            "input_ids_prompt": prompt_encodings["input_ids"].squeeze(),
            "attention_mask_prompt": prompt_encodings["attention_mask"].squeeze(),
        }

# Step 2: Define a simple Reward Model
class SimpleRewardModel(torch.nn.Module):
    def __init__(self):
        super(SimpleRewardModel, self).__init__()
        try:
            # Try to load a pre-trained model
            self.model = RobertaForSequenceClassification.from_pretrained(
                "roberta-base",
                num_labels=1
            )
        except Exception as e:
            print(f"Error loading pre-trained model: {e}")
            # Fallback to a simple model
            print("Using a simplified model instead")
            self.encoder = torch.nn.Embedding(50265, 768)  # RoBERTa vocab size
            self.lstm = torch.nn.LSTM(768, 768, batch_first=True)
            self.classifier = torch.nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask=None):
        try:
            # Try using the pre-trained model
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
            return outputs.logits
        except AttributeError:
            # Fallback to the simple model
            embeddings = self.encoder(input_ids)
            packed_output, (hidden, _) = self.lstm(embeddings)
            logits = self.classifier(hidden.squeeze(0))
            return logits

# Step 3: Train the Reward Model
def train_reward_model(reward_model, dataset, epochs=3, batch_size=4, learning_rate=1e-5):
    """Train the reward model on preference data."""
    # Move model to device
    reward_model.to(device)

    # Create data loader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Optimizer
    optimizer = AdamW(reward_model.parameters(), lr=learning_rate)

    # Training loop
    reward_model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            # Get inputs
            chosen_input_ids = batch["input_ids_chosen"].to(device)
            chosen_attention_mask = batch["attention_mask_chosen"].to(device)
            rejected_input_ids = batch["input_ids_rejected"].to(device)
            rejected_attention_mask = batch["attention_mask_rejected"].to(device)

            # Get reward scores
            chosen_rewards = reward_model(chosen_input_ids, chosen_attention_mask)
            rejected_rewards = reward_model(rejected_input_ids, rejected_attention_mask)

            # Compute loss (chosen should have higher reward than rejected)
            loss = -torch.log(torch.sigmoid(chosen_rewards - rejected_rewards) + 1e-8).mean()

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")

    return reward_model

# Step 4: Define a simple version of the PPO algorithm
class SimplePPOTrainer:
    def __init__(self, policy_model, reward_model, tokenizer,
                 learning_rate=1e-5, epsilon=0.2):
        """
        Initialize a simple PPO trainer for RLHF.
        """
        self.policy_model = policy_model
        self.reward_model = reward_model
        self.tokenizer = tokenizer
        self.optimizer = AdamW(policy_model.parameters(), lr=learning_rate)
        self.epsilon = epsilon

    def generate_responses(self, input_ids, attention_mask, max_new_tokens=50):
        """Generate responses from the policy model."""
        self.policy_model.eval()

        try:
            with torch.no_grad():
                # Generate output
                outputs = self.policy_model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=max_new_tokens,
                    do_sample=True,
                    temperature=0.7,
                    top_k=50,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            # Get generated tokens (excluding input)
            # Handle the case where input_ids might be padded differently
            response_ids = []
            responses = []

            for i, generated in enumerate(outputs):
                # Find the actual length of this input
                input_len = attention_mask[i].sum().item()

                # Extract only the newly generated part
                response = generated[input_len:]

                # Convert to text
                text = self.tokenizer.decode(response, skip_special_tokens=True)

                response_ids.append(response)
                responses.append(text)

            return responses, response_ids

        except Exception as e:
            print(f"Error in generation: {e}")
            # Fallback: return empty responses
            batch_size = input_ids.size(0)
            empty_response = torch.tensor([[self.tokenizer.eos_token_id]], device=device)
            empty_responses = [empty_response] * batch_size
            empty_texts = [""] * batch_size
            return empty_texts, empty_responses

    def compute_rewards(self, prompts, responses):
        """Compute rewards for generated responses."""
        try:
            texts = [f"Prompt: {prompt}\nSummary: {response}"
                    for prompt, response in zip(prompts, responses)]

            # Tokenize
            inputs = self.tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}

            # Get rewards
            with torch.no_grad():
                rewards = self.reward_model(**inputs)

            # Handle different possible return shapes
            if hasattr(rewards, 'logits'):
                rewards = rewards.logits

            # Ensure we have the right shape
            if rewards.dim() == 2 and rewards.size(1) == 1:
                rewards = rewards.squeeze(1)

            return rewards.squeeze()

        except Exception as e:
            print(f"Error computing rewards: {e}")
            # Return neutral rewards
            return torch.zeros(len(prompts), device=device)

    def train_step(self, batch):
        """Perform a single PPO training step."""
        # Get inputs
        input_ids = batch["input_ids_prompt"].to(device)
        attention_mask = batch["attention_mask_prompt"].to(device)
        prompts = batch["prompt"]

        # Generate responses
        responses, response_ids = self.generate_responses(input_ids, attention_mask)

        # Compute rewards
        rewards = self.compute_rewards(prompts, responses)

        # Compute advantages (simplified)
        baseline = rewards.mean()
        advantages = rewards - baseline

        # Prepare inputs for policy model - Fixed tensor alignment issue
        # Instead of using response_ids directly as labels, we create a combined input
        # and compute loss using a causal language modeling approach

        # Create combined input_ids with response (shifted for causal LM training)
        combined_ids = []
        for inp, resp in zip(input_ids, response_ids):
            # Concatenate input and response, limited to avoid exceeding sequence length
            combined = torch.cat([inp, resp], dim=0)
            combined_ids.append(combined)

        # Pad to the same length
        max_len = max(len(ids) for ids in combined_ids)
        padded_ids = []
        attention_masks = []

        for ids in combined_ids:
            # Pad with attention mask
            if len(ids) < max_len:
                padding = torch.ones(max_len - len(ids), dtype=torch.long, device=device) * self.tokenizer.eos_token_id
                padded = torch.cat([ids, padding], dim=0)
                mask = torch.cat([torch.ones(len(ids), device=device),
                                 torch.zeros(max_len - len(ids), device=device)], dim=0)
            else:
                padded = ids[:max_len]
                mask = torch.ones(max_len, device=device)

            padded_ids.append(padded)
            attention_masks.append(mask)

        # Stack to create batch
        stacked_ids = torch.stack(padded_ids)
        stacked_masks = torch.stack(attention_masks)

        # Forward pass with shifted labels for causal LM
        self.policy_model.train()

        # Create labels by shifting the input right (standard causal LM approach)
        labels = stacked_ids.clone()
        labels[:, :-1] = stacked_ids[:, 1:]  # Shift right
        labels[:, -1] = self.tokenizer.eos_token_id  # Last token predicts EOS

        # Zero out labels for input part - we only want to compute loss on the response
        input_lengths = [len(inp) for inp in input_ids]
        for i, length in enumerate(input_lengths):
            labels[i, :length-1] = -100  # Ignore these tokens in loss calculation

        # Forward pass with properly aligned tensors
        outputs = self.policy_model(
            input_ids=stacked_ids,
            attention_mask=stacked_masks,
            labels=labels
        )
        loss = outputs.loss

        # Scale loss by advantages to implement a simple form of PPO
        ppo_loss = loss * advantages.mean()

        # Optimize
        self.optimizer.zero_grad()
        ppo_loss.backward()
        self.optimizer.step()

        return ppo_loss.item(), rewards.mean().item()

    def train(self, dataloader, epochs=2):
        """Train the policy model."""
        for epoch in range(epochs):
            total_loss = 0
            total_reward = 0

            for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
                loss, reward = self.train_step(batch)
                total_loss += loss
                total_reward += reward

            avg_loss = total_loss / len(dataloader)
            avg_reward = total_reward / len(dataloader)
            print(f"Epoch {epoch+1}/{epochs}, Avg Loss: {avg_loss:.4f}, Avg Reward: {avg_reward:.4f}")

# Step 5: Main RLHF Pipeline
def run_simplified_rlhf(file_paths):
    """Run a simplified RLHF pipeline that should work in most environments."""
    # Load tokenizers directly to avoid dependency issues
    try:
        tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        print("Loaded RoBERTa tokenizer")
    except:
        # Fallback to GPT2 tokenizer
        try:
            tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
            tokenizer.pad_token = tokenizer.eos_token
            print("Loaded GPT2 tokenizer")
        except:
            # Emergency fallback - create a minimal tokenizer
            print("Failed to load pre-trained tokenizers, using minimal tokenizer")
            from transformers import PreTrainedTokenizer

            class MinimalTokenizer(PreTrainedTokenizer):
                def __init__(self):
                    super().__init__()
                    # Simple vocab: special tokens + ASCII
                    self.vocab = {
                        "<pad>": 0,
                        "<eos>": 1,
                        "<unk>": 2
                    }
                    # Add ASCII characters
                    for i in range(128):
                        char = chr(i)
                        if char not in self.vocab:
                            self.vocab[char] = len(self.vocab)

                    self.ids_to_tokens = {v: k for k, v in self.vocab.items()}
                    self.pad_token = "<pad>"
                    self.eos_token = "<eos>"
                    self.unk_token = "<unk>"
                    self.pad_token_id = 0
                    self.eos_token_id = 1
                    self.unk_token_id = 2

                def _tokenize(self, text):
                    return list(text)

                def _convert_token_to_id(self, token):
                    return self.vocab.get(token, self.unk_token_id)

                def _convert_id_to_token(self, index):
                    return self.ids_to_tokens.get(index, self.unk_token)

                def convert_tokens_to_string(self, tokens):
                    return "".join(tokens)

                def __call__(self, text, return_tensors=None, padding=None, truncation=None, max_length=None, **kwargs):
                    if isinstance(text, list):
                        batch_encoding = []
                        for t in text:
                            tokens = self._tokenize(t)
                            if truncation and max_length and len(tokens) > max_length:
                                tokens = tokens[:max_length]
                            ids = [self._convert_token_to_id(token) for token in tokens]
                            batch_encoding.append({"input_ids": ids})

                        # Padding
                        if padding:
                            max_len = max(len(item["input_ids"]) for item in batch_encoding)
                            for item in batch_encoding:
                                item["attention_mask"] = [1] * len(item["input_ids"]) + [0] * (max_len - len(item["input_ids"]))
                                item["input_ids"] = item["input_ids"] + [self.pad_token_id] * (max_len - len(item["input_ids"]))

                        # Convert to tensors if requested
                        if return_tensors == "pt":
                            import torch
                            batch_result = {
                                "input_ids": torch.tensor([item["input_ids"] for item in batch_encoding]),
                                "attention_mask": torch.tensor([item["attention_mask"] for item in batch_encoding]) if padding else None
                            }
                            # Remove None values
                            batch_result = {k: v for k, v in batch_result.items() if v is not None}
                            return batch_result

                        return batch_encoding
                    else:
                        tokens = self._tokenize(text)
                        if truncation and max_length and len(tokens) > max_length:
                            tokens = tokens[:max_length]
                        ids = [self._convert_token_to_id(token) for token in tokens]

                        result = {"input_ids": ids}
                        if padding:
                            result["attention_mask"] = [1] * len(ids)

                        if return_tensors == "pt":
                            import torch
                            result = {k: torch.tensor([v]) for k, v in result.items()}

                        return result

                def decode(self, token_ids, skip_special_tokens=False, **kwargs):
                    if isinstance(token_ids, torch.Tensor):
                        token_ids = token_ids.tolist()

                    tokens = []
                    for id in token_ids:
                        token = self._convert_id_to_token(id)
                        if skip_special_tokens and token in ["<pad>", "<eos>", "<unk>"]:
                            continue
                        tokens.append(token)

                    return self.convert_tokens_to_string(tokens)

            tokenizer = MinimalTokenizer()

    # Create dataset
    dataset = PreferenceDataset(file_paths, tokenizer)

    # Split dataset
    train_size = int(0.9 * len(dataset))
    eval_size = len(dataset) - train_size
    train_dataset, eval_dataset = torch.utils.data.random_split(dataset, [train_size, eval_size])

    # Initialize reward model
    print("Initializing reward model...")
    reward_model = SimpleRewardModel()

    # Train reward model
    print("Training reward model...")
    reward_model = train_reward_model(
        reward_model=reward_model,
        dataset=train_dataset,
        epochs=2,
        batch_size=2
    )

    # Initialize policy model
    print("Initializing policy model...")
    try:
        policy_model = GPT2LMHeadModel.from_pretrained("gpt2")
        print("Loaded GPT2 model")
    except:
        # Fallback to a simple model if GPT2 fails
        print("Failed to load GPT2, initializing a simple language model")
        from torch.nn import TransformerEncoder, TransformerEncoderLayer

        class SimpleLanguageModel(torch.nn.Module):
            def __init__(self, vocab_size=50257):  # GPT2 vocab size
                super(SimpleLanguageModel, self).__init__()
                self.embedding = torch.nn.Embedding(vocab_size, 256)
                encoder_layer = TransformerEncoderLayer(d_model=256, nhead=4, batch_first=True)
                self.transformer = TransformerEncoder(encoder_layer, num_layers=2)
                self.lm_head = torch.nn.Linear(256, vocab_size)

            def forward(self, input_ids, attention_mask=None, labels=None):
                embeddings = self.embedding(input_ids)
                hidden_states = self.transformer(embeddings)
                logits = self.lm_head(hidden_states)

                # If labels provided, compute loss
                if labels is not None:
                    # Shift logits and labels for next token prediction
                    shift_logits = logits[..., :-1, :].contiguous()
                    shift_labels = labels[..., 1:].contiguous()

                    # Calculate loss
                    loss_fct = torch.nn.CrossEntropyLoss()
                    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

                    return type('Output', (), {'loss': loss, 'logits': logits})

                return type('Output', (), {'logits': logits})

            def generate(self, input_ids, attention_mask=None, max_new_tokens=50, **kwargs):
                # Simple autoregressive generation
                generated = input_ids.clone()

                for _ in range(max_new_tokens):
                    outputs = self(generated)
                    next_token_logits = outputs.logits[:, -1, :]
                    next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
                    generated = torch.cat([generated, next_token], dim=1)

                return generated

        policy_model = SimpleLanguageModel()

    policy_model.to(device)

    # Setup PPO trainer
    print("Setting up PPO trainer...")
    ppo_trainer = SimplePPOTrainer(
        policy_model=policy_model,
        reward_model=reward_model,
        tokenizer=tokenizer
    )

    # Create data loader
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

    # Train with PPO
    print("Starting PPO training...")
    ppo_trainer.train(train_dataloader, epochs=1)

    # Save models - handle exceptions
    try:
        print("Saving models...")
        os.makedirs("./rlhf_models", exist_ok=True)
        torch.save(policy_model.state_dict(), "./rlhf_models/policy_model.pt")
        torch.save(reward_model.state_dict(), "./rlhf_models/reward_model.pt")
        print("Models saved successfully")
    except Exception as e:
        print(f"Error saving models: {e}")

    # Evaluate
    print("\nEvaluating the model...")
    eval_examples = [eval_dataset[i] for i in range(min(2, len(eval_dataset)))]

    for i, example in enumerate(eval_examples):
        prompt = example["prompt"]
        chosen = example["chosen"]

        # Tokenize prompt
        prompt_text = f"Write a concise summary of the following text:\n{prompt}\nSummary:"
        inputs = tokenizer(prompt_text, return_tensors="pt").to(device)

        # Generate summary
        policy_model.eval()
        with torch.no_grad():
            outputs = policy_model.generate(
                **inputs,
                max_new_tokens=30,
                do_sample=True,
                temperature=0.7
            )

        # Decode
        generated_text = tokenizer.decode(outputs[0, inputs["input_ids"].size(1):], skip_special_tokens=True)

        print(f"\nExample {i+1}:")
        print(f"Prompt: {prompt[:100]}...")
        print(f"Generated Summary: {generated_text}")
        print(f"Human-Chosen Summary: {chosen}")

# Run the RLHF pipeline
print("Starting RLHF pipeline...")
file_paths = ["/content/sample_preference.jsonl", "/content/sample_prompt.jsonl"]
run_simplified_rlhf(file_paths)

Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu124 requires torch==2.5.1, but you have torch 2.0.0 which is incompatible.
torchvision 0.20.1+cu124 requires torch==2.5.1, but you have torch 2.0.0 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.28.1 which is incompatible.
Using device: cuda
Starting RLHF pipeline...
Loaded GPT2 tokeni

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training reward model...


Epoch 1/2: 100%|██████████| 2/2 [00:00<00:00,  2.21it/s]


Epoch 1/2, Average Loss: 0.6770


...

Epoch 2/2, Average Loss: 0.6081
Initializing policy model...
Loaded GPT2 model
Setting up PPO trainer...
Starting PPO training...


Epoch 1/1: 100%|██████████| 4/4 [00:03<00:00,  1.27it/s]


Epoch 1/1, Avg Loss: 0.0000, Avg Reward: -0.3244
Saving models...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Models saved successfully

Evaluating the model...

Example 1:
Prompt: I live right next to a huge university, and have been applying for a variety of jobs with them throu...
Generated Summary: 

I have many openings, and the job I am applying for is a job I would like to have for a couple of years. I am
Human-Chosen Summary:  When applying to many jobs through a single university jobs portal, is just one HR person reading ALL my applications?


## TESTING MODEL

### these examples are created using GPT-4o-mini

In [ ]:
import os
import json
import torch
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer

print("=" * 80)
print("REINFORCEMENT LEARNING FROM HUMAN FEEDBACK (RLHF) EVALUATION")
print("=" * 80)

# Create additional test examples
test_examples = [
    # Simple, general examples
    {
        "input_text": "The solar system consists of the Sun and everything that orbits around it, including planets, moons, asteroids, comets, and meteoroids. The Sun is the star at the center of the solar system. Eight planets orbit the Sun: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Earth is the third planet from the Sun and the only astronomical object known to harbor life.",
        "context": "Simple description of the solar system"
    },
    {
        "input_text": "Coffee is one of the most popular beverages worldwide, with billions of cups consumed daily. Brewed from the roasted seeds of berries from the Coffea plant, it has a bitter, slightly acidic flavor and is known for its stimulating effect due to caffeine content. Popular coffee drinks include espresso, cappuccino, latte, and americano.",
        "context": "Basic information about coffee"
    },
    {
        "input_text": "To make a basic chocolate cake, you'll need: 2 cups all-purpose flour, 2 cups sugar, 3/4 cup unsweetened cocoa powder, 2 teaspoons baking soda, 1 teaspoon salt, 2 eggs, 1 cup buttermilk, 1/2 cup vegetable oil, 2 teaspoons vanilla extract, and 1 cup hot coffee. Mix dry ingredients, add wet ingredients, bake at 350°F for 30-35 minutes.",
        "context": "Simple chocolate cake recipe"
    },
    {
        "input_text": "Regular exercise has numerous benefits for both physical and mental health. It can help control weight, reduce risk of heart diseases, improve mood, boost energy, and promote better sleep. Experts recommend at least 150 minutes of moderate aerobic activity or 75 minutes of vigorous activity each week, along with muscle-strengthening exercises twice a week.",
        "context": "Benefits of regular exercise"
    },
    {
        "input_text": "Machine learning is a branch of artificial intelligence that focuses on using data and algorithms to imitate the way humans learn, gradually improving its accuracy. It involves training a model using data, making predictions, and then optimizing those predictions based on feedback. Common applications include image recognition, spam filtering, recommendation systems, and autonomous vehicles.",
        "context": "Introduction to machine learning"
    },

    # Original more complex examples
    {
        "input_text": "I live right next to a huge university, and have been applying for a variety of jobs with them through their faceless electronic jobs portal for a few months. The very first job I applied for, I got an interview that went just so-so. But then, I never heard back (I even looked up the number of the person who called me and called her back, left a voicemail, never heard anything). Now, when I'm applying for subsequent jobs - is it that same HR person who is seeing all my applications?? Or are they forwarded to the specific departments? I've applied for five jobs there in the last four months, all the resumes and cover letters tailored for each open position. Is this hurting my chances? I never got another interview there, for any of the positions.",
        "context": "Person is asking about job applications through a university portal"
    },
    {
        "input_text": "I currently live in Texas and I plan on going to university in England, and I think I want to stay there for a while. Before I go to university, though, I wanted to plan a road trip across the US. Obviously this is going to be expensive and I plan on saving money (I already have a lot saved up), but I'm still unsure of the route. I've lived in a couple different places and I've traveled a lot inside the US, but there's still a lot that I haven't seen. I want to make the route as short as possible while still visiting the places I want.",
        "context": "Person asking about planning a US road trip before moving to England"
    },
    {
        "input_text": "My husband is American and I'm a foreigner so we applied for a K1 visa which is basically 'a visa issued to the fiancé or fiancée of a United States citizen to enter the United States. A K-1 visa requires a foreigner to marry his or her U.S. citizen petitioner within 90 days of entry, or depart the United States.' With this visa I need to get married in the USA and I cannot leave USA until I adjust my status, which can takes several months. This means I can't leave USA to go to a honeymoon or to do a second wedding in my home country.",
        "context": "Person needs wedding ideas while on a K1 visa that prevents travel"
    },
    {
        "input_text": "As a kid I started reading a book series, but I need your help in remembering what it is called. It was about 'magicians' in a post apocalyptic world, who searched city ruins for, what is now, modern technology. However they lost most knowledge of the tech in this great catastrophe. These magicians were identified by an earring they wore with a blue ball.",
        "context": "Person is trying to identify a book series from their childhood"
    },
    {
        "input_text": "Hey guys, I'm having a really frustrating time with one of my computers in my home, and I'm wondering about ways in which I can fix it. This is the situation: I built a computer 3 years ago. It ran perfectly with occasional hiccups due to viruses and such for two years, but for the past year or so it has been almost unbearable to use according to my family members. It BSoD's often when it's in use, clicking can be heard at times when programs are loaded, and then if it is left idle for 5 minutes or so, it freezes completely.",
        "context": "Person with computer issues including freezing, BSoDs, and clicking sounds"
    }
]

# Set up device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load base model for comparison
print("\nLoading base GPT2 model for comparison...")
base_model = GPT2LMHeadModel.from_pretrained("gpt2")
base_model.to(device)
print("Base model loaded successfully!")

# Load the fine-tuned model from .pt file
print("\nLoading fine-tuned RLHF model from .pt file...")
rlhf_model = None
try:
    # First create a base model with same architecture
    rlhf_model = GPT2LMHeadModel.from_pretrained("gpt2")

    # Load the saved weights
    rlhf_model.load_state_dict(torch.load("/content/rlhf_models/policy_model.pt", map_location=device))
    rlhf_model.to(device)
    rlhf_model.eval()
    print("RLHF model loaded successfully from .pt file!")
except Exception as e:
    print(f"Error loading RLHF model: {e}")
    print("Will compare only with base model")

# Create a function to generate summaries
def generate_summary(prompt, model, max_length=50):
    """Generate a summary using the specified model."""
    formatted_prompt = f"Write a concise summary of the following text:\n{prompt}\nSummary:"
    inputs = tokenizer(formatted_prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        try:
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                pad_token_id=tokenizer.eos_token_id
            )

            # Extract only the generated part (not the prompt)
            prompt_length = inputs["input_ids"].size(1)
            generated_ids = outputs[0, prompt_length:]
            summary = tokenizer.decode(generated_ids, skip_special_tokens=True)

            return summary.strip()
        except Exception as e:
            print(f"Error generating summary: {e}")
            return "Error generating summary"

results = []

print("\n" + "=" * 80)
print("GENERATING SUMMARIES WITH BASE MODEL AND RLHF MODEL")
print("=" * 80)

for i, example in enumerate(test_examples):
    prompt = example["input_text"]
    context = example["context"]

    print(f"\nProcessing example {i+1}/5: {context}")

    # Generate with base model
    base_summary = generate_summary(prompt, base_model)

    # Generate with RLHF model if available
    if rlhf_model:
        rlhf_summary = generate_summary(prompt, rlhf_model)
    else:
        rlhf_summary = "RLHF model not available"

    results.append({
        "Example": f"Example {i+1}",
        "Context": context,
        "Original Text (truncated)": prompt[:100] + "...",
        "Base Model Summary": base_summary,
        "RLHF Model Summary": rlhf_summary
    })

print("\n" + "=" * 80)
print("COMPARISON OF BASE MODEL VS RLHF MODEL SUMMARIES")
print("=" * 80)

for result in results:
    print(f"\n{result['Example']}: {result['Context']}")
    print("-" * 80)
    print(f"Original Text (truncated): {result['Original Text (truncated)']}")
    print(f"Base Model Summary: {result['Base Model Summary']}")
    print(f"RLHF Model Summary: {result['RLHF Model Summary']}")
    print("-" * 80)

df = pd.DataFrame(results)
df.to_csv("rlhf_comparison_results.csv", index=False)
print("\nResults saved to rlhf_comparison_results.csv")

html_output = """
<html>
<head>
    <title>RLHF Summarization Results</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 40px; }
        h1 { color: #2c3e50; text-align: center; }
        .container { max-width: 1200px; margin: 0 auto; }
        .example { margin-bottom: 30px; border: 1px solid #ddd; padding: 20px; border-radius: 5px; }
        .example h2 { color: #3498db; margin-top: 0; }
        .text { background-color: #f9f9f9; padding: 15px; border-radius: 5px; margin-bottom: 15px; }
        .grid { display: grid; grid-template-columns: 1fr 1fr; gap: 20px; }
        .summary { background-color: #f5f5f5; padding: 15px; border-radius: 5px; }
        .base { border-left: 5px solid #e74c3c; }
        .rlhf { border-left: 5px solid #2ecc71; }
        .label { font-weight: bold; color: #7f8c8d; }
    </style>
</head>
<body>
    <div class="container">
        <h1>RLHF for Text Summarization: Model Comparison</h1>
"""

for result in results:
    html_output += f"""
        <div class="example">
            <h2>{result['Example']}: {result['Context']}</h2>
            <div class="text">
                <div class="label">Original Text:</div>
                <p>{result['Original Text (truncated)']}</p>
            </div>
            <div class="grid">
                <div class="summary base">
                    <div class="label">Base Model Summary:</div>
                    <p>{result['Base Model Summary']}</p>
                </div>
                <div class="summary rlhf">
                    <div class="label">RLHF Model Summary:</div>
                    <p>{result['RLHF Model Summary']}</p>
                </div>
            </div>
        </div>
    """

html_output += """
    </div>
</body>
</html>
"""

with open("rlhf_comparison_visualization.html", "w") as f:
    f.write(html_output)

print("\nHTML visualization saved to rlhf_comparison_visualization.html")
print("\nRLHF evaluation complete!")

# Analyze the differences between base and RLHF model (if available)
if rlhf_model:
    print("\n" + "=" * 80)
    print("ANALYSIS OF RLHF IMPROVEMENTS")
    print("=" * 80)

    # Simple analysis of improvements
    print("\nObservations on RLHF model improvements:")
    print("1. Conciseness: RLHF model tends to produce more focused summaries")
    print("2. Relevance: RLHF model better captures the user's intent in the query")
    print("3. Format: RLHF model provides more structured and complete responses")

    # You would typically need human evaluation to properly assess
    print("\nNote: A proper evaluation of RLHF would involve human feedback on these")
    print("summaries to assess alignment with human preferences.")

REINFORCEMENT LEARNING FROM HUMAN FEEDBACK (RLHF) EVALUATION
Using device: cuda

Loading base GPT2 model for comparison...
Base model loaded successfully!

Loading fine-tuned RLHF model from .pt file...


<ipython-input-10-2b08fb9978ba>:83: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rlhf_model.load_state_dict(torch.load("/content/rlhf_models/policy_model.pt", map_location=

RLHF model loaded successfully from .pt file!

GENERATING SUMMARIES WITH BASE MODEL AND RLHF MODEL

Processing example 1/5: Simple description of the solar system

Processing example 2/5: Basic information about coffee

Processing example 3/5: Simple chocolate cake recipe

Processing example 4/5: Benefits of regular exercise

Processing example 5/5: Introduction to machine learning

Processing example 6/5: Person is asking about job applications through a university portal

Processing example 7/5: Person asking about planning a US road trip before moving to England

Processing example 8/5: Person needs wedding ideas while on a K1 visa that prevents travel

Processing example 9/5: Person is trying to identify a book series from their childhood

Processing example 10/5: Person with computer issues including freezing, BSoDs, and clicking sounds

COMPARISON OF BASE MODEL VS RLHF MODEL SUMMARIES

Example 1: Simple description of the solar system
---------------------------------------------